In [1]:
import sys
sys.path.append('/home/trukhinmaksim/src')
import json

In [2]:
from time import time
from src.utils.CacheAdapter import FlatAdapter, EXP_END_OF_DATA, createAdapter_02_04_25_GOOD, createTrainSetAdapter_02_04_25_GOOD, createTestSetAdapter_02_04_25_GOOD, createNormAdapter_02_04_25_GOOD
from src.utils.CacheAdapter import Factory_21_04_25_HIGH as AdapterFactory_21_04_25
from src.utils.Corpus import CacheCorpus, FlatCorpus, Factory_21_04_25_HIGH as CorpusFactory_21_04_25
from src.utils.DatasetManager import ProjectsDatasetManager

In [3]:
def clearFiles():
    with open("/home/trukhinmaksim/src/data/train_02-04-25/train_02-04-25", "w") as file:
        pass
    with open("/home/trukhinmaksim/src/data/train_02-04-25/test_02-04-25", "w") as file:
        pass
#clearFiles()

In [3]:
def createMainCorpus():
    return CorpusFactory_21_04_25.createNormCorpus()#JSONMultiFileAdapter(CACHE_FILE_NAME)
    #manager = ProjectsDatasetManager(50, cacheAdapter = adapter)

trainsetAdapter = AdapterFactory_21_04_25.createTrainSetAdepter() #createTrainSetAdapter_02_04_25_GOOD()
testsetAdapter = AdapterFactory_21_04_25.createTestSetAdepter() #createTestSetAdapter_02_04_25_GOOD()

In [5]:
DATASET_SIZE = 482555

testsetSize = int(DATASET_SIZE * .15)
trainsetSize = int(DATASET_SIZE * .85)

print(testsetSize)
print(trainsetSize)

72383
410171


In [ ]:
def showTagsDistrubutionForCorpus(corpus):
    tagsCount = {}
    tagsLst = []

    def sort(tagsCount):
        print("Sorting tags")
        tagsLst = sorted([*tagsCount.items()], key = lambda pair_1: pair_1[1], reverse = True)

        return tagsLst


    start = time()
    i = 0
    try:
        for proj in corpus:
            if i % 100000 == 0: 
                print(f"Scanned {i} projects in {time() - start} s")
            for tag in proj.tags:
                if tag in tagsCount:
                    tagsCount[tag] += 1
                else:
                    tagsCount[tag] = 1

            i += 1
    except Exception as exp:
        raise exp
    finally:
        tagsLst = sort(tagsCount)
        tagsCount.clear()
    
        return tagsLst

In [14]:
from src.utils.helpers import getTagsQuantitiesForCorpus

corpus = createMainCorpus()
#corpus = CorpusFactory_21_04_25.createFlatCorpus()
tagsLst = getTagsQuantitiesForCorpus(corpus)

working_list = tagsLst[:180] # 116

working_list

Scanned 0 projects in 5.841255187988281e-05 s
Scanned 100000 projects in 0.9019854068756104 s
Scanned 200000 projects in 1.819814920425415 s
Scanned 300000 projects in 2.7445759773254395 s
Scanned 400000 projects in 3.608790874481201 s
Sorting tags


[('Python', 69247),
 ('JavaScript', 67274),
 ('javascript', 45278),
 ('python', 39661),
 ('react', 37967),
 ('TypeScript', 28506),
 ('typescript', 26678),
 ('nodejs', 26092),
 ('HTML', 25168),
 ('reactjs', 24556),
 ('Jupyter Notebook', 20938),
 ('C++', 19168),
 ('html', 18509),
 ('Java', 18291),
 ('java', 16782),
 ('C#', 15427),
 ('PHP', 14513),
 ('Go', 14093),
 ('nextjs', 13648),
 ('android', 13498),
 ('machine-learning', 13389),
 ('css', 13144),
 ('tailwindcss', 12069),
 ('mongodb', 11743),
 ('hacktoberfest', 11214),
 ('C', 11210),
 ('api', 11067),
 ('CSS', 10729),
 ('Shell', 9487),
 ('Dart', 9399),
 ('flutter', 9233),
 ('Kotlin', 8970),
 ('deep-learning', 8931),
 ('php', 8374),
 ('golang', 7958),
 ('express', 7816),
 ('redux', 7408),
 ('firebase', 6975),
 ('ai', 6617),
 ('expressjs', 6223),
 ('python3', 6184),
 ('postgresql', 6049),
 ('mysql', 6030),
 ('Vue', 6002),
 ('Rust', 5942),
 ('html5', 5609),
 ('django', 5597),
 ('linux', 5588),
 ('react-native', 5579),
 ('game', 5565),
 ('i

In [15]:
def countProjectsWithTagsToRemove(corpus):
    counter = 0
    tagsNamesToRemove = set((p[0] for p in working_list))

    for doc in corpus:
        tags = tagsNamesToRemove & set(doc.tags[2:])

        if len(tags): counter += 1

    return counter

total_working_proj_with_tags = countProjectsWithTagsToRemove(corpus)
total_working_proj_with_tags

461033

In [16]:
from math import ceil
tagsToRemove = {}

k = 1.17 # special parameter, must be adjusted to collect right amount of documents into the testing set 
for item in working_list:
    pair = list(item)
    pair[1] = ceil(k * pair[1] / total_working_proj_with_tags * testsetSize)

    tagsToRemove[pair[0]] = pair[1]

In [17]:
tagsNamesToRemove = set(tagsToRemove.keys())

[*tagsToRemove.items()][-10:]

[('serverless', 314),
 ('spring', 309),
 ('redux-toolkit', 309),
 ('vuejs', 300),
 ('fastapi', 299),
 ('data', 298),
 ('youtube', 297),
 ('socket-io', 296),
 ('eslint', 295),
 ('r', 294)]

In [18]:
def decreaseTags(tagsToRemove, tags):
    for tag in tags:
        tagsToRemove[tag] -= 1

def keepDoc(tagsToRemove, tags): 
    for tag in tags:
        if tagsToRemove[tag] == 0:
            return True

    # if all tags can be removed, then False
    return False

def splitSets(corpus):
    ignoreDocs = set()
    testSetCounter = 0
    i = 0

    for doc in corpus:
        tags = tagsNamesToRemove & set(doc.tags[1:]) # tags[0] is an id, shouldn't be viewed

        if len(tags):
            # if project contains tags, that are in 'working list'

            if not keepDoc(tagsToRemove, tags):
                # write to the testing set
                testsetAdapter.save([{"tokens" : doc.words, "tags" : doc.tags}])
                ignoreDocs.add(doc.tags[0])

                if len(ignoreDocs) >= testsetSize:
                    print(f"Collected {len(ignoreDocs)} on iteration {i}, test set updated")
                    break
                decreaseTags(tagsToRemove, tags)
        i += 1

    corpus.reset()
    #for doc in corpus: pass # to exhaust the corpus

    counter = 0
    for doc in corpus:
        if doc.tags[0] not in ignoreDocs:
            trainsetAdapter.save([{"tokens" : doc.words, "tags" : doc.tags}])
            counter += 1

    print(f"Train set updated with {counter} documents")

splitSets(corpus)

Collected 72383 on iteration 79641, test set updated
Train set updated with 410172 documents


In [ ]:
# TODO: finish that class for corpus separation
class CorpusSeparator:
    class EXP_PROPORTION_WRONG(Exception):
        def __init__(self):
            super().__init__("Wrong proportion specified, train and test sets sizes portions must add up to 100")
    
    def __init__(self, mainCorpus, trainAdapter, testAdapter, trainPortion = 85, testPortion = 15):
        # will take all data from mainCorpus and separate it into train and test set, according to proportions
        # algorithm = "balanced" | "random" | "first n"
        #     "balanced" will automatically determine, how many project with each tag should be moved into the test set, more projects with popular tags will be moved to test set, less projects with unpopular tags will be moved

        if trainPortion + testPortion != 100: raise EXP_PROPORTION_WRONG

        corpusLen = 0
        for doc in mainCorpus:
            corpusLen += 1

        testsetSize = int(corpusLen * testPortion / 100)
        trainsetSize = int(corpusLen * trainPortion / 100)


    def balanced(self):
        tagsCount = {}
        tagsLst = []

        def sort(tagsCount):
            print("Sorting tags")
            tagsLst = sorted([*tagsCount.items()], key = lambda pair_1: pair_1[1], reverse = True)

            return tagsLst

        start = time()
        i = 0
        try:
            for proj in corpus:
                if i % 100000 == 0: 
                    print(f"Scanned {i} projects in {time() - start} s")
                for tag in proj.tags:
                    if tag in tagsCount:
                        tagsCount[tag] += 1
                    else:
                        tagsCount[tag] = 1

                i += 1
        except Exception as exp:
            raise exp
        finally:
            tagsLst = sort(tagsCount)
            tagsCount.clear()

            working_list = tagsLst[:120]

        counter = 0
        tagsNamesToRemove = set((p[0] for p in working_list))

        for doc in corpus:
            tags = tagsNamesToRemove & set(doc.tags[2:])

            if len(tags): counter += 1

        return counter

        total_working_proj_with_tags = countProjectsWithTagsToRemove(corpus)
    
    def separate(self, algorithm = "balanced"):
        pass

In [19]:
def checkCorpusesSizes():
    trainCorpus = FlatCorpus(trainsetAdapter)
    testCorpus = FlatCorpus(testsetAdapter)

    for corp, name in ((trainCorpus, "Train"), (testCorpus, "Test")):
        c = 0
        for doc in corp:
            c += 1
        print(f"{name} contains {c}")

In [20]:
checkCorpusesSizes()
#showTagsDistrubutionForCorpus(trainCorpus)[:50]

Train contains 410172
Test contains 72383
